# Emissions Calculation Methodology

##### Step 1: Determine the operating mode of the vessel at each AIS data point

GPS Speed provides movement activity which can be used to categorize vessels as either moving or stationary. 

There are four operating modes:
* Transit: If speed > 1 OR navigational status = 0, 3, 4, 8, 11, 12 (considered 'moving' statuses) and it's OUTSIDE the maneuvering zone but inside the bigger port fence.
* Anchorage: The vessel must be at an anchorage and non-moving (speed <1).
* Maneuvering: If speed > 1 or navigational status = 0, 3, 4, 8, 11, 12 (considered 'moving' statuses) and it's INSIDE the maneuvering zone.
* Alongside/Hotel: The vessel must be berthed (within a berth polygon) and speed <1. Note: If speed is > 1, the mode is either transit or manoeuvre.
When determining moving vessels, either speed or navigation status can be used.

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("vessel_movements/vessel_movements_PPT.csv")
df.head()

,imo,mmsi,vessel_id,vessel_name,date_of_build,vessel_type,group,timestamp,date_only,lon,...,anchorage,terminal,maneuvering_zone,p,vref,sfc_me,sfc_ae,sfc_ab,ael,abl
0,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:01:42.000Z,2023-09-17,103.826065,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.0,230.0,300.0,507.0,365.0
1,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:01:44.000Z,2023-09-17,103.826065,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.0,230.0,300.0,507.0,365.0
2,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:07:43.000Z,2023-09-17,103.825690,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.0,230.0,300.0,507.0,365.0
3,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:07:45.000Z,2023-09-17,103.825690,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.0,230.0,300.0,507.0,365.0
4,9223253,525119038,692017,PRIMA TANGGUH LVI,2002-04-01,Crude/Oil Products Tanker,Crude & Products Tanker,2023-09-17T00:10:42.000Z,2023-09-17,103.825580,...,PPT Anchorage,NaN,NaN,7860.0,14.5,185.0,230.0,300.0,507.0,365.0


In [ ]:
def operating_modes(speed, nav_stat, maneuver_zone, anchorage, berth):
    if speed > 1 and nav_stat in [0, 3, 4, 8, 11, 12] and maneuver_zone is np.nan:
        return "Transit"
    elif anchorage != np.nan and speed < 1:
        return "Anchorage"
    elif speed < 1 and nav_stat in [0, 3, 4, 8, 11, 12] and maneuver_zone is not np.nan:
        return "Maneuvering"
    elif berth is not np.nan and speed < 1:
        return "Alongside/Hotel"
    else:
        return "Other"


##### Step 2: Log events at each timestamp

The event duration is the difference between two consecutive timestamps for each vessel i.e. same IMO number. 

Note: If the difference between two AIS time stamps is more than 10,000 seconds, then the event is not continuous and should not undergo calculation for this greater than 24-hour period. Thereafter it should start afresh.


##### Step 3: Determine the vessel type

Determine if the vessel is an Ocean Going Vessel (OGV, of below types) or a different type.

1. Bulk carrier - cargo vessel for the carriage of dry bulk cargo
2. Container ship – cargo vessel for the carriage of containerized goods
3. General cargo – cargo vessel for the carriage of various types of dry cargo
4. LPG / Anhydrous ammonia tanker – cargo vessel for the bulk carriage of liquefied petroleum gas or ammonia
5. Passenger / Cruise ship - vessels for the carriage of passengers.
6. Product tanker – cargo vessel for the carriage of refined oil products carried in bulk
7. Reefer ship - special form of cargo ship. It is used to transport goods that must remain refrigerated during the entire transport or where the cold chain must not be interrupted
8. Ro-ro ship – cargo vessel for the carriage of wheeled cargo
9. Chemical Tankers - cargo vessels for the carriage of Chemicals

NOTE: LNG Tankers are to be excluded


##### Step 4: Determine Emissions Tier

For OGVs, identify its Emissions Tier using Model Year (derived from the vessel's build date).

##### Step 5: Determine Fuel Category

Ports have defined fuel categories (e.g. High Sulphur, Global 0.5%, ECA). 

For 2023, Singapore will fall under Global 2020.

##### Step 6: Retrieve specific ship details for all vessels that are within the port boundary

1. Main engine SFC (sfc_me)
2. Aux engine SFC (sfc_ae)
3. Auxiliary Boiler SFC (sfc_ab) 


##### Step 7: Calculate Emissions

This step is dependent on the previous steps. Refer to below explanation for emission calculations

Note: 
* Sequence to be strictly followed for calculations
* Emissions do not need to be calculated for data points with missing parameters (e.g. p, speed, ael, abl, sfc_me, sfc_ae) and can be zero as default, where these parameters are used in calculation formulas. 
* If the missing parameters are not used in the calculation formula (e.g. for calculating emissions in alongside mode), there should be emissions.





###### Step 7a: Get the following parameters from the vessel movements data:

* p
* Reference speed (vref)
* Actual speed (AS)


###### Step 7b: Calculate maximum speed

Formula:

>Maximum Speed (MS) = 1.066 x vref

###### Step 7c: Get activity hours (A) for each operating mode

This was calculated in Step 2. 


###### Step 7d (i): Calculate Engine Load Factor (LF) for propulsion engines

 >\\(LF =(AS/MS){^3}\\)
<br>Where</br> 
  LF = Load Factor
  <br>AS =  Actual speed (knots)</br>
    MS = Maximum speed (knots)

###### Step 7d(ii): Look up Low Load Adjustment Factor (LLAF) using LF calculated in Step 7d(i)

* Refer to llaf_table
* LLAF is not applicable if LF is more than 0.2. LLAF will be 1 in such cases.


###### Note: Propulsion engine emission can only be calculated for transit and maneuver modes since at other times the main engine is not running.

###### Step 7e: If LF < 0.2 and mode is transit or maneuver, then LF will be defaulted to 0 .2.
Note: this default is not applied when identifying LLAF


###### Step 7f: Emission factor (EF) table is provided.  Singapore is a non-Emission Controlled Area (ECA) so Global 2020 rows are applicable. The EF table has some formulas applied for certain pollutants:
<br>
1. NOx

* Refer to tables Propulsion Engine Emission Factors and Auxiliary Engine Emission Factors for main engine and auxiliary engine. The values used are dependent on Engine Type and Emissions Tier in vessel movements table.

* Do the same for Auxiliary Boiler (Boiler Engine Emission Factors table).

2. PM10 will be calculated as follows:
* Main Engine and Auxiliary Engine = 0.23 + (sfc_me or sfc_ae ) x 7 x 0.02247 x (0.005-0.0024)
* Boiler = direct look up to Boiler Engine Emission Factors table

3. PM2.5 = 0.92 x PM10

4. VOC = direct look up EngineType column in vessel movements table.

5. SOx – Refer to vessel movements table.
* For Main Engine= sfc_me x 2 x 0.97753 x 0.005
* For Auxiliary engine= sfc_ae x 2 x 0.97753 x 0.005
* For Auxiliary Boiler= sfc_ab x 2 x 0.97753 x 0.005

6. CO2 - Refer to vessel movements table
* For Main Engine= sfc_me x 0.867 x 3.667
* For Auxiliary engine= sfc_ae x 0.867 x 3.667
* For Auxiliary Boiler= sfc_ab x 0.867 x 3.667

7. CH4 - Refer to vessel movements table
* For Main Engine and Auxiliary Boiler = corresponding direct look up Propulsion Engine Emission Factors and Auxiliary Engine Emission Factors with reference to Columns EngineType and EmissionsTier
* For Aux engine = Dependent on Auxiliary engine load (Column ael) and EmissionsTier, direct look-up

8. N2O – EngineType and EmissionsTier, dependent look up


###### Step 7g:	Based on above identifications, the formulae below will be used to get the final results.

The emission factor is in terms of emissions per unit of energy from the engine. It is multiplied by the energy consumed while in each mode.
<br></br>
Propulsion engine emissions for each mode are calculated according to the general equation:
<br></br>
>E = P x LF x A x EF x LLAF
<br></br>
Where
<br>E = Emissions (grams [g])</br>
P = Maximum Continuous Rating (MCR) Power for propulsion engines (kilowatts [kW]) 
<br>LF = Load Factor (percent of vessel's total power)</br>
A = Activity (hours [h]) (hours/call)
<br>EF = Emission Factor (grams per kilowatt-hour [g/kWh])</br>
LLAF = Low Load Adjustment Factor for when the load factor is below 20%.

Auxiliary engine emissions are calculated according to the general equation:
<br></br>
>E = LA X A x EF
<br></br>
Where
<br>E = Emissions (grams [g])</br>
\\(L{_A}\\) = Auxiliary engine load (kW)
<br>A = Activity (hours [h]) (hours/call)</br>
EF = Emission Factor (grams per kilowatt-hour [g/kWh])
<br></br>

Auxiliary boiler emissions are calculated using the general equation below:
<br></br>
>E = LB X A x EF
<br></br>
Where
<br>E = Emissions (grams [g])</br>
\\(L{_B}\\) = Boiler Load (kW),
<br>A = Activity (hours [h]) (hours/call)</br>
EF = Emission Factor (grams per kilowatt-hour [g/kWh])

###### Step 7h:	In vessel movements data, identify the below:

\\(L{_A}\\) = Auxiliary Engine load (Column ael)
<br>
\\(L{_B}\\) = Auxiliary Boiler load (Column abl)

#####Note: 
*Total emissions of a certain pollutant type = Sum of emissions from Main Engine, Auxiliary engine and Auxiliary Boiler

*Focus is not calculation of any other pollutants outside this discussion.
<br></br>


#### Total Electrical Load Analysis

Alongside or hotel load is the electrical load when the vessel is at the terminal/berth. In this mode the Main Engine is off since the Propulsion is not happening. Auxiliary Engines are running since they need to supply the electrical load for the ship and loading/discharging operating machineries.
<br></br>
Auxiliary Boilers produce steam and hence cannot be plugged into the electrical grid and there is no machinery onboard to generate steam.
<br></br>
Hence, for the purpose of this scope the Auxiliary engine load (column ael) in alongside mode will be used to identify the electrical load demand of the ships.
<br></br>
Total electrical demand from ship in alongside mode (Tael) = time-weighted average electrical load (ael) in alongside mode = (Sum of Product of Ael multiplied by alongside duration hours) divide by (Sum of total alongside duration in hours).


#### Load demand variation from the grid through day and night on a monthly basis

1. For each “Alongside” operation mode, identify the time of the day as per below definition of Day and Night from timestamp column in vessel movements table.
* Day = 0900 to 1700 hours
* Night = 1700 to 0900 hours
2. Create visualizations to analyze the trends of the daily and monthly demand on the shore supply grid.
3. For DayEL and NightEL results, use the formula provided in previous section for the associated time description stated above.
<br></br>
NOTE: This day/ night demand can vary significantly depending on the weather and time of year in many parts of the world if not so much in Singapore. Example, winter may lead to smaller daylight time and hence more demand for vessel’s external lights to be on and vice versa. 

